In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
03_train_logreg.py
------------------
Trains Logistic Regression for SMS spam:
  TF-IDF (uni+bi-grams) -> RandomOverSampler -> LogisticRegression
Grid-search on (C, min_df) with 5-fold CV; evaluate on VAL.
Saves the trained pipeline to ../OUTPUT/logreg.joblib and a JSON report.

Assumes:
  ../DATA/splits/train.csv
  ../DATA/splits/val.csv
  Columns: Label, SMS_Message
"""
from pathlib import Path
import json
import pandas as pd
import joblib

from imblearn.pipeline import Pipeline
from imblearn.over_sampling import RandomOverSampler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score

# Paths
splits_dir = Path("../DATA/splits")
out_dir = Path("../OUTPUT/03_Split_and_Train")
out_dir.mkdir(parents=True, exist_ok=True)

model_path = out_dir / "logreg.joblib"
report_path = out_dir / "train_report.json"

# Load data
train = pd.read_csv(splits_dir / "train.csv")
val = pd.read_csv(splits_dir / "val.csv")

X_train = train["SMS_Message"].tolist()
y_train = (train["Label"].str.lower() == "spam").astype(int).values

X_val = val["SMS_Message"].tolist()
y_val = (val["Label"].str.lower() == "spam").astype(int).values

# Pipeline: TF-IDF -> ROS -> Logistic Regression
pipe = Pipeline([
    ("tfidf", TfidfVectorizer(ngram_range=(1, 2), min_df=2)),
    ("ros", RandomOverSampler(random_state=42)),
    ("clf", LogisticRegression(max_iter=1000, random_state=42))
])

# Small grid
param_grid = {
    "tfidf__min_df": [1, 2, 3],
    "clf__C": [0.5, 1.0, 2.0, 4.0],
    "clf__penalty": ["l2"],
    "clf__solver": ["lbfgs"],
}

# Optimize F1 on spam (positive class)
gs = GridSearchCV(pipe, param_grid, scoring="f1", cv=5, n_jobs=-1, verbose=1)
gs.fit(X_train, y_train)
best_model = gs.best_estimator_

# Validate on VAL (original distribution)
y_pred = best_model.predict(X_val)
y_prob = best_model.predict_proba(X_val)[:, 1]

prec, rec, f1, _ = precision_recall_fscore_support(
    y_val, y_pred, average="binary", zero_division=0
)
auc = roc_auc_score(y_val, y_prob)

report = {
    "best_params": gs.best_params_,
    "val_metrics": {
        "precision": float(prec),
        "recall": float(rec),
        "f1": float(f1),
        "roc_auc": float(auc),
    },
}

# Save artifacts
joblib.dump(best_model, model_path)
with open(report_path, "w") as f:
    json.dump(report, f, indent=2)

print("Saved model:", model_path)
print("Saved report:", report_path)
print("Best params:", gs.best_params_)
print("VAL metrics:", report["val_metrics"])


Fitting 5 folds for each of 12 candidates, totalling 60 fits
Saved model: ../OUTPUT/03_Split_Dataset/logreg.joblib
Saved report: ../OUTPUT/03_Split_Dataset/train_report.json
Best params: {'clf__C': 4.0, 'clf__penalty': 'l2', 'clf__solver': 'lbfgs', 'tfidf__min_df': 2}
VAL metrics: {'precision': 0.9904761904761905, 'recall': 0.9285714285714286, 'f1': 0.9585253456221198, 'roc_auc': 0.9934145619573795}
